In [2]:
# !pip install pandas
# !pip install selenium
# !pip install tqdm
# !pip install multiprocess


# Gathering data

### Importing relevant packages

In [2]:
import pandas as pd
import time

from multiprocess import Pool, cpu_count
from random import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from tqdm.auto import tqdm

### Gathering all the links that we will scrape from RA

In [2]:
chrome_options = Options()  
chrome_options.headless = True
driver = webdriver.Chrome(options=chrome_options)

all_links = []
from tqdm.auto import tqdm
for page in tqdm(range(2001,5001)):
    url = f'https://www.reclameaqui.com.br/empresa/pagseguro/lista-reclamacoes/?pagina={page}'
    driver.get(url)
    all_links += [complain.get_attribute('href') for complain in driver.find_elements_by_class_name('link-complain-id-complains')]

In [6]:
with open('30-40-50k.txt', 'w') as filehandle:
    for listitem in all_links:
        filehandle.write('%s\n' % listitem)

NameError: name 'all_links' is not defined

### Consolidating all the urls gathered from the step above

In [3]:
content_list = []

file_list = ['first10k.txt', '10-20k.txt', '30-40-50k.txt']
for file in file_list:
    my_file = open(file, "r")
    content = my_file.read()
    content_list += content.split("\n")
len(content_list)        

49283

### Gathering relevant data (title, status, etc.) from each url

In [6]:
chrome_options = Options()
chrome_options.headless= True
driver = webdriver.Chrome(options = chrome_options)

urls = content_list[40001:]
counta = 30000
df = pd.DataFrame(columns = ['id', 'caption', 'details', 'status', 'complaint'])
troubled_links = []
good_links = []

for url in tqdm(urls):
#     print(url)
    try:
        driver.get(url)
    except:
        pass
    counta +=1
    time.sleep(random()+1)
    try:
        good_links.append({'id' : [counta],
         'caption' : [driver.find_element_by_xpath(
            '//*[@id="complain-detail"]/div/div[1]/div[2]/div/div[1]/div[2]/div[1]/h1').text],
         'details': [driver.find_element_by_xpath(
            '//*[@id="complain-detail"]/div/div[1]/div[2]/div/div[1]/div[2]/div[1]/ul[1]').text],
         'status' : [driver.find_element_by_xpath(
            '//*[@id="complain-detail"]/div/div[1]/div[2]/div/div[1]/div[2]/div[3]/span[2]/strong').text],
         'complaint' : [driver.find_element_by_xpath(
            '//*[@id="complain-detail"]/div/div[1]/div[2]/div/div[2]/p').text]})
#         df = pd.concat([df_load, df])
    except:
        print(f'ID {counta} deu ruim')
        troubled_links.append(url)
        pass
    
    if (counta) % 40 == 0:
        df = pd.DataFrame(good_links)
        df.to_excel('./backup/backup_' + str(counta) + '.xlsx')
        good_links = []
        driver.close()
        chrome_options = Options()
        chrome_options.headless= True
        driver = webdriver.Chrome(options = chrome_options)

driver.close()


ID 32734 deu ruim
ID 34031 deu ruim
ID 37095 deu ruim
ID 37658 deu ruim
ID 37948 deu ruim
ID 38999 deu ruim
ID 39028 deu ruim



In [4]:
df.reset_index(drop=True)

,id,caption,details,status,complaint
